# HOSPITAL PATIENT RECORDS to OMOP

This is mainly mapped to visit occurrence following: https://docs.google.com/spreadsheets/d/1Vw1Dr2K4oG__cDQTutGaJhZvGUvQTLwc4qWreP6qMSs/edit?gid=571393688#gid=571393688

We still need visit_concept_id

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
df_hospitalPatientRecords_IDEA4RC = pd.read_csv("./IDEA4RC-data/hospitalPatientsRecordIDEA4RC.csv")
df_hospitalPatientRecords_IDEA4RC.head(5)

### Hospital Patient Records to Visit Occurrence

In [ ]:
conn_params = {
    'dbname': 'omopdb',
    'user': 'postgres',
    'password': 'mysecretpassword',
    'host': 'omop_postgres',
    'port': '5432'
}

try:
    conn = psycopg2.connect(**conn_params)
    cur = conn.cursor()
    
    cur.execute("SELECT current_database();")
    current_db = cur.fetchone()[0]
    print("Base de datos actual:", current_db)
    
    cur.execute("SELECT table_schema, table_name FROM information_schema.tables WHERE table_name = 'visit_occurrence';")
    tables = cur.fetchall()
    if not tables:
        print("Table does not exist")
    else:
        print("Table found in:", tables)
    
except Exception as e:
    print("Ocurrió un error:", e)

### From Hospital Patient Records to Visit Occurrence

In [ ]:
df_table = pd.DataFrame()
df_table.index = range(len(df_hospitalPatientRecords_IDEA4RC))
sql = """
    INSERT INTO public.visit_occurence (person_id,care_site_id,visit_start_date,visit_concept_id)
    VALUES (%s,%s,%s)
"""
sqlGetCareSite = """
    SELECT care_site_id FROM care_site WHERE care_site_name = %s
"""
for index, row in df_hospitalPatientRecords_IDEA4RC.iterrows():
    cur.execute(sqlGetCareSite, (row['Hospital']))
    care_site_id = cur.fetchone()[0]
    visit_concept_id=???
    cur.execute(sql, (row['Patient'],care_site_id,row['Date of first contact with the hospital'],visit_concept_id))
    conn.commit()

cur.close()